In [ ]:
import tensorflow
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential # Подлючаем класс создания модели Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D # Подключаем класс Dense - полносвязный слой, и Flatten - данные выстраиваем в линию, одномерные
from tensorflow.keras.optimizers import Adam # Подключаем оптимизатор Adam
from tensorflow.keras import utils #Утилиты для to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix # для построения матрицы ошибок
from sklearn.preprocessing import StandardScaler # для нормализации данных
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import numpy as np # Подключаем библиотеку numpy
import pandas as pd # Библиотека pandas

# from google.colab import files #Для загрузки своей картинки

import matplotlib.pyplot as plt #Отрисовка изображений
import seaborn as sns # seaborn, надстройка визуализации над matplotlib
from PIL import Image #Отрисовка изображений

#Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline

<p style="border:3px #ff0000  solid;">
Задача: Создать нейронную сеть

<p style="border:3px #ff0000  solid;">
1. Загружаем данные

In [ ]:
pima_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_3.1/pima-indians-diabetes.csv', header=None)
pima_df.columns = ['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'class']
pima_df.sample(5)

<p style="border:3px #ff0000  solid;">
2. Подготавливаем данные (приводим их к нужным типам данных, проводим базовое EDA, делаем  выводы, если возможно).

In [ ]:
pima_df.info()

In [ ]:
#Преобразуем типы данных в нужный формат
pima_df = pima_df.astype('float64')

In [ ]:
pima_df.describe().T

In [ ]:
pima_df['class'].value_counts()

<p style="border:3px #00B344  solid;">
Пропуски отсутсвуют. Типы данных соответствуют. Данные готовы к дальнейшей работе. Наблюдается дисбаланс целевого признака, который необходимо учесть в дальнейшей работе.
</p>

<p style="border:3px #ff0000  solid;">
3. Создаем тренировочную и валидационную выборки.

In [ ]:
target_pima = pima_df['class']
features_pima = pima_df.drop(['class'], axis=1)
#Разделяем датасет на тренировочную и валидационную части
features_train, features_test, target_train, target_test = train_test_split(features_pima, target_pima, test_size=0.3, random_state=45)

<p style="border:3px #ff0000  solid;">
4. Масштабируем данные с помощью StandardScaler из библиотеки Scikit-learn.

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_sc = scaler.transform(features_train)
features_valid_sc = scaler.transform(features_test)

<p style="border:3px #ff0000  solid;">
5. Создайте функцию, при вызове которой будет собираться и компилироваться (model.compile) архитектура модели. Функция должна возвращать собранную архитектуру. Параметры слоев и их количество вы выбираете сами, стараясь добиться наилучшего качества.

In [ ]:
def get_sequential_model(layers, input_dim = 8, dense = 10, optimazer = "adam", activation = "relu"):
    model = Sequential()
    model.add(Dense(dense, input_dim=input_dim, activation=activation)) #Входной слой
    for i in range(1, layers):
        model.add(Dense(dense, activation=activation)) #Внутренние слои
    model.add(Dense(1, activation="sigmoid")) #Выходной слой
    model.compile(loss="binary_crossentropy", optimizer=optimazer, metrics=["accuracy"]) # Компилируем модель
    return model

<p style="border:3px #ff0000  solid;">
6. Обучите модель.

In [ ]:
#Так как имеется дисбаланс классов, посчитаем веса классов
class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(target_train), y=target_train)
class_weight = dict(enumerate(class_weight))

In [ ]:
num_epoch = 15
dense = 15
layers_model = 5 #Количество внутренних слоев в модели
input_dim_model = 8 #Количесвто входных данных
sequential_model = get_sequential_model(layers_model, input_dim_model, dense, "adam", "relu")
history = sequential_model.fit(features_train_sc, target_train, batch_size=128, epochs=num_epoch, verbose=0, class_weight=class_weight, validation_data=(features_valid_sc, target_test)) # !!!!!!Добавим параметр class_weight=class_weight, так как имеется дисбаланс

<p style="border:3px #ff0000  solid;">
7. Оцените качество модели по выбранной вами метрике (выбор метрики должен быть обоснован).

    Так как модель решает задачу бинарной классификации, для оценки качества модели выберем метрику accuracy

In [ ]:
val_loss, vall_acc = sequential_model.evaluate(features_valid_sc, target_test)

<p style="border:3px #ff0000  solid;">
8. Постройте графики точности и графики ошибки в зависимости от количества эпох для тренировочной выборки и для валидационной.

In [ ]:
#График точности
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('График точности')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
#График ошибки
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('График ошибки')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.show()

<p style="border:3px #ff0000  solid;">
9. Постройте матрицу ошибок для валидационной выборки.

In [ ]:
target_test_pred = sequential_model.predict(features_valid_sc)
target_test_pred = (target_test_pred > 0.5).astype(int)

# пробуем построить матрицу ошибок
plt.figure(figsize=(5, 5)) # задаем размеры всего рисунка
cm = confusion_matrix(target_test, target_test_pred) # создаём матрицу ошибок
sns.heatmap(cm, annot=True, fmt='d', cmap='RdBu')
plt.ylabel('true label')
plt.xlabel('predicted label')
plt.show()

<p style="border:3px #00B344  solid;">
Максимальное качество модели с выбранными параметрами составило от 0.68 до 0.71. Модель получилась достаточно низкого качества. Требуется оптимизация выбранных параметров модели.

<p style="border:3px #ff1010  solid;">
==============ЗАДАНИЕ ПРО==========================

<p style="border:3px #ff1010  solid;">
1. Установите библиотеку scikeras, либо найдите как импортировать модуль KerasClassifier в tensorflow.

<p style="border:3px #ff1010  solid;">
2. Из этой же библиотеки scikeras импортируйте модуль KerasClassifier (найдите в документации как его импортировать).

<p style="border:3px #ff1010  solid;">
3. Поместите вашу модель в оболочку KerasClassifier.

In [ ]:
kerasClassifier_model = KerasClassifier(model=get_sequential_model, layers = None, input_dim = None, dense = None, optimazer = None, activation = None, verbose=0)

<p style="border:3px #ff1010  solid;">
4. Подберите оптимальные параметры для модели с помощью GridSearchCV или RandomizedSearchCV.

        Подберем оптимальный вариант модели с помощью GridSearchCV

In [ ]:
params = {
    'layers' : [2, 4, 6, 8],
    'input_dim' : [8],
    'dense' : [128, 256, 512],
    'optimazer': ['adam', 'sgd'],
    'activation' : ['relu', 'softmax']
}

In [ ]:
grid_search = GridSearchCV(estimator=kerasClassifier_model, param_grid=params, cv=5, scoring = 'accuracy')
grid_result = grid_search.fit(features_train_sc, target_train)

In [ ]:
#Наилучшие параметры модели:
best_params = grid_result.best_params_
print("Наилучшие параметры модели: ", best_params)

<p style="border:3px #ff1010  solid;">
5. Посчитайте метрику с наилучшими параметрами, напишите наилучшие параметры для модели в выводе.

In [ ]:
predict_from_grid = grid_result.predict(features_valid_sc)
accuracy_score(target_test, predict_from_grid)

    Проверим полученное значение точности на модели, обученной вручную

In [ ]:
# Наилучшие параметры модели: {'activation': 'relu', 'dense': 512, 'input_dim': 8, 'layers': 2, 'optimazer': 'adam'}
optimized_model = get_sequential_model(2, 8, 512, "adam", "relu")
hist = optimized_model.fit(features_train_sc, target_train, batch_size=128, epochs=15, verbose=0, class_weight=class_weight, validation_data=(features_valid_sc, target_test)) # !!!!!!Добавим параметр class_weight=class_weight, так как имеется дисбаланс
print("Точность на тренировочной выборке: ", max(hist.history['accuracy']))

In [ ]:
#График точности
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('График точности')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
#График ошибки
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('График ошибки')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.show()

    Построим матрицу ошибок для валидационной выборки.

In [ ]:
optim_target_test_pred = optimized_model.predict(features_valid_sc)
optim_target_test_pred = (optim_target_test_pred > 0.5).astype(int)

# пробуем построить матрицу ошибок
plt.figure(figsize=(5, 5)) # задаем размеры всего рисунка
cm = confusion_matrix(target_test, optim_target_test_pred) # создаём матрицу ошибок
sns.heatmap(cm, annot=True, fmt='d', cmap='RdBu')
plt.ylabel('true label')
plt.xlabel('predicted label')
plt.show()

In [ ]:
accuracy_score(target_test, optim_target_test_pred)

<p style="border:3px #00B344  solid;">
Вывод:
</p>

    Значения метрики accuracy для различных моделей:
    Первоначальная модель sequential_model = от 0.68 до 0.71
    predict_from_grid = 0.74 (KerasClassifier и GridSearchCV)
    optimized_model = 0.74 (модель KerasClassifier, обученная вручную с подобранными параметрами)
    
    Вцелом, в результате проведенной оптимизации и подбора параметров, удалось добиться некоторого улучшения качества модели.
    Наилучшие параметры модели: {'activation': 'relu', 'dense': 512, 'input_dim': 8, 'layers': 2, 'optimazer': 'adam'}

P.S.
При перезапуске ядра могут быть получены немного другие значения accuracy и оптимпльных параметров модели.